In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
import joblib

# =============================
# 🔹 Load Dataset
# =============================
file_path = 'dataset_sdn.csv'
df = pd.read_csv(file_path, encoding='latin1')

# Identify numeric and categorical features
numerical_features = [feature for feature in df.columns if df[feature].dtypes != 'O' and feature != "label"]
print("The number of numerical features is", len(numerical_features), "and they are : \n", numerical_features)
categorical_features = [feature for feature in df.columns if df[feature].dtypes == 'O']
print("The number of categorical features is", len(categorical_features), "and they are : \n", categorical_features)
target = 'label'

# Separate features and target variable
X = df[numerical_features + categorical_features]
y = df[target]

# Split into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# =============================
# 🔹 Handle Missing Values
# =============================
# Impute missing values in numeric features
num_imputer = SimpleImputer(strategy='mean')
X_train_numeric = num_imputer.fit_transform(X_train[numerical_features])
X_test_numeric = num_imputer.transform(X_test[numerical_features])

# Impute missing values in categorical features
cat_imputer = SimpleImputer(strategy='most_frequent')
X_train_categorical = cat_imputer.fit_transform(X_train[categorical_features])
X_test_categorical = cat_imputer.transform(X_test[categorical_features])

# =============================
# 🔹 Standardize Numeric Features
# =============================
scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

# =============================
# 🔹 Encode Categorical Features
# =============================
vectorizer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat_encoded = vectorizer.fit_transform(X_train_categorical)
X_test_cat_encoded = vectorizer.transform(X_test_categorical)

# =============================
# 🔹 Combine Processed Features
# =============================
X_train_final = np.hstack((X_train_numeric_scaled, X_train_cat_encoded))
X_test_final = np.hstack((X_test_numeric_scaled, X_test_cat_encoded))

# Convert target to NumPy arrays
y_train = y_train.values
y_test = y_test.values

# =============================
# 🔹 Helper Function for Evaluation
# =============================
def evaluate_model(y_true, y_pred, model_name):
    print(f"Evaluation for {model_name}:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(classification_report(y_true, y_pred))
    conf_matrix = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = conf_matrix.ravel()
    print(f"TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
    print("\n")


# =============================
# 🔹 Logistic Regression
# =============================
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train_final, y_train)
y_pred_lr = lr_model.predict(X_test_final)
evaluate_model(y_test, y_pred_lr, "Logistic Regression")

# Save Logistic Regression model
joblib.dump(lr_model, 'Modells/LogisticRegression_model.pkl')
joblib.dump(scaler, 'Modells/LogisticRegression_scaler.pkl')
joblib.dump(vectorizer, 'Modells/LogisticRegression_vectorizer.pkl')

print(" Logistic Regression model, scaler, and vectorizer saved.")

The number of numerical features is 19 and they are : 
 ['dt', 'switch', 'pktcount', 'bytecount', 'dur', 'dur_nsec', 'tot_dur', 'flows', 'packetins', 'pktperflow', 'byteperflow', 'pktrate', 'Pairflow', 'port_no', 'tx_bytes', 'rx_bytes', 'tx_kbps', 'rx_kbps', 'tot_kbps']
The number of categorical features is 3 and they are : 
 ['src', 'dst', 'Protocol']
Evaluation for Logistic Regression:
Accuracy: 0.8469
              precision    recall  f1-score   support

           0       0.86      0.90      0.88     12712
           1       0.83      0.77      0.80      8157

    accuracy                           0.85     20869
   macro avg       0.84      0.83      0.84     20869
weighted avg       0.85      0.85      0.85     20869

TP: 6288, TN: 11386, FP: 1326, FN: 1869


 Logistic Regression model, scaler, and vectorizer saved.


C:\Users\hp\anaconda3\envs\ml_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
